# Capstone Project - The Battle of the Neighbourhoods
### IBM Data Science Capstone

## Table of Contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

For this project, we will be trying to find the best locations for opening a **Middle Eastern restaurant** in **British Columbia, Canada**. For the sake of this report, the customers for such a restaurant are termed ‘Arabs’.

In order to find the optimal location, we will analyze certain datasets by following criteria such as metropolitan areas that have the **highest population of Arab origin** but the **lowest number of currently existing Middle Eastern restaurants**. This way, the newly opened restaurant will have the best chance to attract the most customers, resulting in good business - provided of course that the food is good.

At the end of this report, using the information found, **a location recommendation for a new restaurant will be suggested**. This will be important information for stakeholders that are looking to open new restaurants, as a business can suffer when there are too many similar restaurants or not enough of the target customers to boost profits.

## Data <a name="data"></a>

In order to approach this problem, two main datasets are needed, obtained from **Statistics Canada** and **Foursquare**. 

The **demographic of each major metropolitan area**, specifically looking at the 2016 Census, will be further sifted by ethnic origins. 
* It is assumed in this report, that **the majority of the Middle Eastern restaurant’s business will come from customers with a Middle Eastern/Arabian background**. 
* When analyzing the ‘Ethnic Origins’ section in the 2016 Census, one of the main ‘Asian origins’ subgroups - ‘West Central Asian and Middle Eastern origins’ - was chosen with the assumption that it includes most of the Arabian countries in Asia, along with other African countries from the ‘African Origins’ group such as Sudan and Libya which speak Arabic.

Additionally, using Foursquare API, the number of **restaurants classified as 'Middle Eastern Restaurants'** for each metropolitan area was found using the **Google Maps API geocoders package** for each location.

## Methodology <a name="methodology"></a>

### Part 1: Obtaining demographic and ethnic information for each Metropolitan Area 

Statistics Canada lists 26 major metropolitan areas (MAs) in British Columbia, Canada. Each of these MAs will go through the process of extracting the required data. For the purpose of this methodology, the city of Abbotsford will be used as an example.

First, on the Statistics Canada website for British Columbia's MAs, the 'Ethnic Origins' dropdown shortcut was used for each MA to obtain a table consisting of countries and the number of people living in that MA that are ethnically from those countries. Statistics Canada has the option to download the table as a CSV file. Once that is done we can then use pandas to read the CSV file into a data frame.

In [19]:
import pandas as pd
df_Abbotsford = pd.read_csv(' ', encoding='latin-1', header = 1)
df_Abbotsford.head()

,Topic,Characteristics,Note,Total,Flag_Total,Male,Flag_Male,Female,Flag_Female,Total.1,Flag_Total.1,Male.1,Flag_Male.1,Female.1,Flag_Female.1
0,NaN,NaN,NaN,Counts (unless otherwise specified),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ethnic origin population,Total - Ethnic origin for the population in pr...,103.0,176330,NaN,87125.0,NaN,89205.0,NaN,4560240.0,NaN,2237490.0,NaN,2322750.0,NaN
2,Ethnic origin population,North American Aboriginal origins,NaN,10875,NaN,5270.0,NaN,5605.0,NaN,302810.0,NaN,146530.0,NaN,156280.0,NaN
3,Ethnic origin population,First Nations (North American Indian),NaN,7000,NaN,3415.0,NaN,3585.0,NaN,220245.0,NaN,106595.0,NaN,113650.0,NaN
4,Ethnic origin population,Inuit,NaN,60,NaN,20.0,NaN,40.0,NaN,2615.0,NaN,1280.0,NaN,1340.0,NaN


As you can see, the table is extensive and requires some cleaning up. First, the columns are renamed in order to easily refer to them. Then all columns and rows were dropped except for those that contained the name of the countries and the total population attributed to them.

In [20]:
df_Abbotsford.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15']

df_Abbotsford.drop(columns=['1','3','5','6','7','8','9','10','11','12','13','14','15'], inplace=True)

df_Abbotsford.drop(index=0, inplace=True)
df_Abbotsford.drop(index=1, inplace=True)

df_Abbotsford.columns= ['Ethnic origin','Abbotsford']

df_Abbotsford.reset_index(inplace = True)

df_Abbotsford.drop(columns=['index'], inplace=True)

df_Abbotsford = df_Abbotsford[:278]

In [21]:
df_Abbotsford.head()

,Ethnic origin,Abbotsford
0,North American Aboriginal origins,10875
1,First Nations (North American Indian),7000
2,Inuit,60
3,Métis,4245
4,Other North American origins,37550


The countries are arranged by Statistics Canada into group and subgroups where the main groups are loosely based on continents. Luckily, there is a subgroup under ‘Asian Origins’ called ‘West Central Asian and Middle Eastern Origins', which contains most of the Arab countries in Asia. Not to forget that Africa also contains Arab countries, under the ‘African Origins’ group, countries such as Egypt and Libya were identified. All of these were added into an array, used to create a data frame containing the population count of Arabs in Abbotsford.

In [24]:
array = ['    West Central Asian and Middle Eastern origins','      Egyptian','      Algerian','      Sudanese','      Moroccan','      Tunisian','      Somali','      Chadian','      Eritrean','      Libyan', '      Djiboutian',]

df_Abbotsford= df_Abbotsford.loc[df_Abbotsford['Ethnic origin'].isin(array)]

In [25]:
df_Abbotsford.head()

,Ethnic origin,Abbotsford
142,Chadian,0
164,Algerian,25
168,Egyptian,60
169,Libyan,0
171,Moroccan,65


This process was repeated for the other twenty-five MAs resulting in twenty-six total data frames. In order to combine then into a single data frame for further analyze, a ‘for loop’ was used to merge them all together.

In [751]:
combine_df = pd.merge(df_Abbotsford, df_Campbell_River)

dflist = [df_Chilliwack, df_Courtenay, df_Cranbrook, df_Dawson_Creek, df_Duncan, df_Fort_St_John, df_Kamloops, df_Kelowna, df_Nanaimo, df_Nelson, df_Parksville, df_Penticton, df_Port_Alberni, df_Powell_River, df_Prince_George, df_Prince_Rupert, df_Quesnel, df_Salmon_Arm, df_Squamish, df_Terrace, df_Vancouver, df_Vernon, df_Victoria, df_Williams_Lake]

for dataframe in dflist:
    combine_df = pd.merge(combine_df,dataframe)

combine_df.head(30) 

,Ethnic origin,Abbotsford,Campbell River,Chilliwack,Courtenay,Cranbrook,Dawson Creek,Duncan,Fort St. John,Kamloops,...,Prince George,Prince Rupert,Quesnel,Salmon Arm,Squamish,Terrace,Vancouver,Vernon,Victoria,Williams Lake
0,Chadian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,0,20,0
1,Algerian,25,0,0,0,0,0,10,0,0,...,0,0,0,0,0,0,615,0,25,0
2,Egyptian,60,20,95,30,0,0,0,30,10,...,55,0,0,0,40,0,3405,30,395,0
3,Libyan,0,0,0,0,0,20,0,0,0,...,0,0,0,0,0,0,210,0,175,0
4,Moroccan,65,0,20,20,10,0,30,20,10,...,10,0,0,0,10,0,1480,0,175,0
5,Sudanese,135,25,25,0,0,0,0,15,0,...,10,0,0,0,0,0,940,0,30,0
6,Tunisian,10,0,0,0,10,0,0,0,10,...,0,0,0,10,0,0,380,0,60,0
7,Djiboutian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Eritrean,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,950,0,65,0
9,Somali,10,0,10,0,0,15,0,10,0,...,0,0,0,0,0,0,2105,0,130,0


Further cleaning was required in order to create a clean data frame, such as renaming certain columns and rows as well as converting certain string values into integers.

In [754]:
name = combine_df.columns[1:]

for column in name:
    combine_df[column] = combine_df[column].str.replace(" ","").astype(int)
    
combine_df = combine_df.append(combine_df.sum(numeric_only=False), ignore_index=True)

combine_df.replace(to_replace='    West Central Asian and Middle Eastern origins', value='Middle Eastern', inplace=True)
combine_df.replace(to_replace='      Chadian      Algerian      Egyptian      Libyan      Moroccan      Sudanese      Tunisian      Djiboutian      Eritrean      Somali    West Central Asian and Middle Eastern origins', value='Total', inplace=True)

In [761]:
combine_df

,Ethnic origin,Abbotsford,Campbell River,Chilliwack,Courtenay,Cranbrook,Dawson Creek,Duncan,Fort St. John,Kamloops,...,Prince George,Prince Rupert,Quesnel,Salmon Arm,Squamish,Terrace,Vancouver,Vernon,Victoria,Williams Lake
0,Chadian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,0,20,0
1,Algerian,25,0,0,0,0,0,10,0,0,...,0,0,0,0,0,0,615,0,25,0
2,Egyptian,60,20,95,30,0,0,0,30,10,...,55,0,0,0,40,0,3405,30,395,0
3,Libyan,0,0,0,0,0,20,0,0,0,...,0,0,0,0,0,0,210,0,175,0
4,Moroccan,65,0,20,20,10,0,30,20,10,...,10,0,0,0,10,0,1480,0,175,0
5,Sudanese,135,25,25,0,0,0,0,15,0,...,10,0,0,0,0,0,940,0,30,0
6,Tunisian,10,0,0,0,10,0,0,0,10,...,0,0,0,10,0,0,380,0,60,0
7,Djiboutian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Eritrean,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,950,0,65,0
9,Somali,10,0,10,0,0,15,0,10,0,...,0,0,0,0,0,0,2105,0,130,0


As nice as this table is, only the last 'Total' column for each MA is required for the next few steps. Therefore, we extract this column using the .loc feature and make a new data frame. We clean up the data frame once more such as arranging the 'Population' column in descending order.

In [764]:
total_df = combine_df.loc[11,:]
total_df = pd.DataFrame (total_df)

total_df.reset_index(inplace=True)

total_df.drop(index=0, inplace=True)
total_df.head()

total_df.columns = ['Metropolitan Area','Population']
total_df.head()

total_df = total_df.sort_values(by='Population', ascending=False)
total_df.reset_index(inplace=True)
total_df.drop(columns=['index'], inplace=True)

In [768]:
total_df

,Metropolitan Area,Population
0,Vancouver,96185
1,Victoria,5750
2,Kelowna,1870
3,Abbotsford,1585
4,Nanaimo,1065
5,Kamloops,555
6,Chilliwack,500
7,Prince George,490
8,Vernon,355
9,Courtenay,305


Eh voila, Part 1 of this report is complete. The above data frame can be saved as a new CSV file so that it can be called on later in Part 3.

### Part 2: Finding the number of currently running Middle Eastern Restaurants in each Metropolitan Area

In this next part, Foursquare API is used to find the number of Middle Eastern restaurants currently running in each MA. First, some basic information required by Foursquare must be outlined before we can get any results such as the category ID for 'Middle Eastern Restaurants' - which can be found on the Foursquare website.

If you remember in Part 1, we analyzed the demographics for twenty-six MAs. However, since we are looking for optimal results, we will only consider the top ten Arab populated MAs. Thus, for Part 2, only the top 10 MAs from Part 1 were run through the Foursquare API program.

In [231]:
address = 'Abbotsford, British Columbia'

geolocator = Nominatim(user_agent="my_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

49.0521162 -122.3294792


In [235]:
search_query = 'Middle Eastern Restaurant'
radius = 17500
categoryId = '4bf58dd8d48988d115941735'
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, categoryId, radius, LIMIT)

results = requests.get(url).json()

food_category = '4d4b7105d754a06374d81259'

arabic_restaurant_categories = ['4bf58dd8d48988d115941735']#, '5bae9231bedf3950379f89e1', '5bae9231bedf3950379f89e7', '56aa371be4b08b9a8d573529', '5744ccdfe4b0c0459246b4ca', '58daa1558bbb0b01f18ec1cd','52e81612bcbc57f1066b79f7', '5bae9231bedf3950379f89e4', '5bae9231bedf3950379f89da', '5bae9231bedf3950379f89ea']

results = requests.get(url).json()['response']['groups'][0]['items']


venues = [(item['venue']['name'],\
                   item['venue']['id'],\
                   (item['venue']['categories']),\
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),\
                  (item['venue']['location']),\
                   item['venue']['location']['distance']) for item in results]        
venues = pd.DataFrame (venues)
venues.columns = ['Venue ID','Venue Name', 'Category', 'Long/Lat', 'Location', 'Distance from MA']
venues.head()

,Venue ID,Venue Name,Category,Long/Lat,Location,Distance from MA
0,Donairo's,4c7c084a3badb1f7eaa55654,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...","(49.04644091693317, -122.29382872581482)","{'address': '2455 McCallum Road', 'crossStreet...",2676
1,Donair Zone,4f513bf1e4b023845d1b1048,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...","(49.050177, -122.338589)","{'address': '160 - 2655 Clearbrook Rd', 'lat':...",698


In [238]:
venues.drop (columns = ['Category', 'Long/Lat', 'Location', 'Distance from MA'], inplace=True)
venues.insert(0, 'MA', 'Abbotsford')
venues_total = pd.DataFrame(venues.shape)
venues_total.drop(index=1, inplace=True)
venues_total.columns = ['Abbotsford']
venues_total.index = ['Total # of Restaurants']
venues_total

,Abbotsford
Total # of Restaurants,2


In [241]:
venues_total.reset_index(inplace = True)

shape = venues_total.loc[0,'Abbotsford']
venues = venues.append({'MA' : 'Abbotsford' , 'Venue ID' : 'N/A', 'Venue Name' : shape} , ignore_index=True)
venues.head()

,MA,Venue ID,Venue Name
0,Abbotsford,Donairo's,4c7c084a3badb1f7eaa55654
1,Abbotsford,Donair Zone,4f513bf1e4b023845d1b1048
2,Abbotsford,N/A,2


So here, in the above dataframe, we see that Abbotsford only has two restaurants classifed as Middle Eastern by Foursquare. We will repeat this process for the other 25 MAs and save them as a csv to be called on in Part 3.So here, in the above data frame, we see that Abbotsford only has two restaurants classified as Middle Eastern by Foursquare. We will store this information in a separate CSV file to be used in just a bit. We will repeat this process for the other twenty-five MAs.

Before we move on to Part 3, we need to create one clean data frame with all the data collected so far in Part 2. As we have been saving each mini data frame for each MA as a CSV in a specific folder, we can use the ‘glob’ module to load them all into one data frame.

In [56]:
import pandas as pd

import glob

path = r''

all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:

    df = pd.read_csv(filename, index_col=None, header=0)

    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [57]:
frame

,Unnamed: 0,MA,Venue ID,Venue Name
0,0,Abbotsford,Donairo's,4c7c084a3badb1f7eaa55654
1,1,Abbotsford,Donair Zone,4f513bf1e4b023845d1b1048
2,2,Abbotsford Total,NaN,2
3,0,Chilliwack Total,NaN,1
4,0,Courtenay Total,NaN,0
5,0,Kamloops,Shawarma Time,5db62d349faee50008689adc
6,1,Kamloops Total,NaN,1
7,0,Kelowna,Basha Donair & Shwarma,5d71c3fbf662f800085b37ab
8,1,Kelowna Total,NaN,7
9,0,Nanaimo,Aladdin's Café,4c6e07c865eda093f19350d0


For Part 3, we will only need the totals for each MA - so why did we keep the ‘Venue Name’ and ‘Venue ID’ columns? Since we set the radius for the Foursquare API to 17500m around each MA, there may be overlap with other MAs. Fortunately, each venue has a unique ‘Venue ID’, so we can check this data frame for duplicate values in the ‘Venue ID’ column.

In [58]:
duplicate = frame[frame.duplicated(['Venue Name'])]
duplicate

,Unnamed: 0,MA,Venue ID,Venue Name
10,1,Nanaimo Total,NaN,1
12,1,Prince George Total,NaN,1
75,1,Vernon Total,NaN,1
83,7,Victoria Total,NaN,7


As can be seen, the only duplicate entries are from the totals, which means that no duplicate Venue IDs were found - great! We change all the NaN values to zero, so that we can use the .loc method to select only the rows that contain zeros (only the total rows have this as their value in the 'Venue ID' column).

In [61]:
frame = frame.fillna(0)
frame.head()

,Unnamed: 0,MA,Venue ID,Venue Name
0,0,Abbotsford,Donairo's,4c7c084a3badb1f7eaa55654
1,1,Abbotsford,Donair Zone,4f513bf1e4b023845d1b1048
2,2,Abbotsford Total,0,2
3,0,Chilliwack Total,0,1
4,0,Courtenay Total,0,0


In [62]:
totals = frame.loc[frame['Venue ID'] == 0]
totals

,Unnamed: 0,MA,Venue ID,Venue Name
2,2,Abbotsford Total,0,2
3,0,Chilliwack Total,0,1
4,0,Courtenay Total,0,0
6,1,Kamloops Total,0,1
8,1,Kelowna Total,0,7
10,1,Nanaimo Total,0,1
12,1,Prince George Total,0,1
73,60,Vancouver Total,0,60
75,1,Vernon Total,0,1
83,7,Victoria Total,0,7


Once again - we clean up the data frame...

In [67]:
totals = totals.drop(columns=['Unnamed: 0', 'Venue ID'])
totals.columns = ['MA', '# of R']
totals = totals.replace('Total', '', regex=True)

totals = totals.reindex([7,9,4,0,5,3,1,6,8,2])
totals.reset_index(inplace = True)
totals = totals.drop(columns=['index'])
totals

,MA,# of R
0,Vancouver,60
1,Victoria,7
2,Kelowna,7
3,Abbotsford,2
4,Nanaimo,1
5,Kamloops,1
6,Chilliwack,1
7,Prince George,1
8,Vernon,1
9,Courtenay,0


Again, we save it to a new CSV just for easy purposes and move onto Part 3!

### Part 3: Putting it all together and finding an optimal location

In this final part, the goal is to put together what we have found in Part 1 and 2. First, we concatenate the two final data frames from Part 1 and 2.

In [71]:
result = pd.concat([first, totals], axis=1)
result = result.drop(columns=['MA'])
result

,Metropolitan Area,Population,# of R
0,Vancouver,96185,60
1,Victoria,5750,7
2,Kelowna,1870,7
3,Abbotsford,1585,2
4,Nanaimo,1065,1
5,Kamloops,555,1
6,Chilliwack,500,1
7,Prince George,490,1
8,Vernon,355,1
9,Courtenay,305,0


Now we have a data frame that contains the top 10 MAs in British Columbia that are most populated by Arabs, along with the number of current Middle Eastern Restaurants open. However, one other factor to account before we draw any conclusions is the population density of each MA. It is important to take into consideration the population of Arabs in an MA relative to the total population of an MA (found again using Statistics Canada), as it will reflect how sparse or dense the target customers are. This is the next step…

In [73]:
result.insert(2, 'Population Density', '', allow_duplicates=False)

each_density = [2426235, 357690, 190565, 176330, 101985, 98850, 100755, 85135, 59715, 53120]

for density in each_density:
    result ['Population Density'] = result['Population']/each_density 

result

,Metropolitan Area,Population,Population Density,# of R
0,Vancouver,96185,0.039644,60
1,Victoria,5750,0.016075,7
2,Kelowna,1870,0.009813,7
3,Abbotsford,1585,0.008989,2
4,Nanaimo,1065,0.010443,1
5,Kamloops,555,0.005615,1
6,Chilliwack,500,0.004963,1
7,Prince George,490,0.005756,1
8,Vernon,355,0.005945,1
9,Courtenay,305,0.005742,0


Now that we have that, all that is left is to find the ratio between the population density of an MA and the number of current restaurants. Ideally, we would like a high population density and a low number of restaurants currently running, therefore we expect the most favourable ratio to be high and sort it as such to reflect that.

In [77]:
ratio = result['Population Density']/result['# of R']
result.insert(4, 'Ratio', ratio, allow_duplicates=False)
result = result.sort_values(by=['Ratio','Population Density'], ascending=False)

result

,Metropolitan Area,Population,Population Density,# of R,Ratio
9,Courtenay,305,0.005742,0,inf
4,Nanaimo,1065,0.010443,1,0.010443
8,Vernon,355,0.005945,1,0.005945
7,Prince George,490,0.005756,1,0.005756
5,Kamloops,555,0.005615,1,0.005615
6,Chilliwack,500,0.004963,1,0.004963
3,Abbotsford,1585,0.008989,2,0.004494
1,Victoria,5750,0.016075,7,0.002296
2,Kelowna,1870,0.009813,7,0.001402
0,Vancouver,96185,0.039644,60,0.000661


## Results and Discussion <a name="results"></a>

Using the above data, it would seem that the top three MAs to open the restaurant would be Courtenay, Nanaimo, and Vernon. While it would simple to say that stakeholders should choose Courtenay as the optimal location, there is a little more analysis that can be done.

Courtenay has the highest ratio due to the fact that there are no Middle Eastern restaurants located there according to Foursquare, so it would seem at first to be an amazing opportunity for a stakeholder - they would be the first Middle Eastern restaurant in Courtenay. However, while 305 potential customers sounds appealing, we must once again check the population density of the top MAs. Even though Nanaimo is ranked second, its population density is almost double that of Courtenay, and its population of Arabs and therefore customers, is three and a half times more. Therefore, it would seem as though Nanaimo is also a pretty good bet for opening a restaurant.

Therefore, even though further analysis using other factors will be needed in order to pinpoint an exact location and address, it would seem as though the general optimal location to open up a Middle Eastern restaurant would be in between Courtenay and Nanaimo.

## Conclusion <a name="conclusion"></a>

The purpose of the project was to identify the most optimal locations for opening up a Middle Eastern restaurant in a metropolitan area in British Columbia, Canada. We went about doing this by analyzing and finding an MA with the highest Arab population in order to have the greatest customer availability, and with the lowest number of already opened Middle Eastern restaurants in order to reduce competition that would otherwise steal customers.

Of course, this analysis points to having the restaurant as mentioned earlier, in between the MAs of Courtenay and Nanaimo, however, further analysis on other factors such as proximity to city centres, price of rent, living costs...etc, will have to be taken into account in order to pinpoint an exact location within this general optimal area.